# Removing Radial Outliers Demonstration
Written By Marty Dewitt October 2023

In [1]:
from math import exp, sqrt, floor
from random import random

import matplotlib.pyplot as plt

# Global variables
GRID_SIZE = 1024
X_CENTER, Y_CENTER = 512, 512

###########
# THIS PART IS ONLY FOR SIMULATING A DIFFRACTION IMAGE
###########

# For testing, simulate a diffraction image on a 1024x1024 grid
# First, simulating a radial distribution (just adding 3 Gaussians bc it's easy)
# Then extending that to a 2D isotropic image
def simulate_radial_distribution():
    def gauss(x, x_center, sigma, intensity):
        return intensity * exp(-(x-x_center)**2 / (2 * sigma**2)) 
    
    gaussian_params = [
        [0, 25, 100], # [x_center, sigma, intensity]
        [150, 25, 50],
        [300, 25, 25]
    ]

    radial_range = 512
    # Add together the intensities of each Gaussian at each radial value
    radial_dist = [sum([gauss(r, *g_param) for g_param in gaussian_params]) for r in range(radial_range)]

    return radial_dist

def simulate_image(radial_distribution):
    """Convert the 1D simulated radial distribution into a 2D isotropic image"""

    image = [[0 for Y in range(GRID_SIZE)] for X in range(GRID_SIZE)]

    for Y in range(GRID_SIZE):
        for X in range(GRID_SIZE):
            R = sqrt((X - X_CENTER)**2 + (Y - Y_CENTER)**2) # Calculate radial value of pixel
            # Calculate the weighted average of the radial distribution intensity at that point
            R_lower = floor(R)
            R_upper = R_lower + 1
            if R_lower >= len(radial_distribution):
                image[Y][X] = 0 # R value is out of bounds, default to 0
            elif R_upper >= len(radial_distribution):
                image[Y][X] = radial_distribution[R_lower] # R value is just outside of bounds, default to edge value
            else:
                # Calculate weighted average
                image[Y][X] = (R - R_lower) * radial_distribution[R_upper] + (R_upper - R) * radial_distribution[R_lower]
            
            # Add some noise to image
            if random() < 0.01:
                image[Y][X] *= 3
    
    return image 


SIMULATED_IMAGE = simulate_image(simulate_radial_distribution())

###########
# END OF SECTION FOR SIMULATING IMAGE
###########



In [3]:
def get_average(array):
    """Return the average and standard deviation of an array"""
    Sum = 0
    Sum_squared = 0
    for element in array:
        if element == None:
            continue # Skip empty values
        Sum += element
        Sum_squared += element**2
    length = len(array)
    if length == 0:
        return 0, 0
    average = Sum / length 
    stdev = sqrt(Sum_squared / length - average**2) # sigma^2 = <X^2> - <X>^2
    return average, stdev

def get_radial_distribution(image):
    """Get average radial intensity after eliminating outliers"""
    radial_range = int(min(X_CENTER, Y_CENTER, GRID_SIZE-X_CENTER, GRID_SIZE-Y_CENTER))
    radial_values = [[] for r in range(radial_range)]

    # First get all pixel values for each radial position as an array that we can average over later
    # It will look like
    #   [ (R = 0 values) [...],
    #     (R = 1 values) [...],
    #     (R = 2 values) [...], etc ]

    for Y in range(GRID_SIZE):
        for X in range(GRID_SIZE):
            R = sqrt((X - X_CENTER)**2 + (Y - Y_CENTER)**2)
            R_index = round(R)
            if R_index < len(radial_values):
                radial_values[R_index].append(image[Y][X])
    
    # So now we have an array where, for each R value, we have a sub-array of each pixel intensity at that R
    # We want to first calculate the average and standard deviation
    # Then eliminate all outliers that are >3sigma away from the average
    # Then re-calculate the average with all outliers removed

    radial_averages, radial_stdevs = [], []
    for r_values in radial_values:
        avg, stdev = get_average(r_values)
        # Create new list with outliers removed
        new_r_values = [r if abs(avg - r) <= 3*stdev else None for r in r_values]
        new_avg, new_stdev = get_average(new_r_values)
        radial_averages.append(new_avg)
        radial_stdevs.append(new_stdev)
    
    return radial_averages, radial_stdevs

def clean_up_image(image):
    """After finding average radial values, replace all outlier pixels with the average value at that radius"""
    radial_avgs, radial_stdevs = get_radial_distribution(image)

    new_image = [[0 for Y in range(GRID_SIZE)] for X in range(GRID_SIZE)]

    for Y in range(GRID_SIZE):
        for X in range(GRID_SIZE):
            R = sqrt((X - X_CENTER)**2 + (Y - Y_CENTER)**2)
            # Calculate the interpolated value for the average and stdev at this R
            # (Same thing I do in simulate_image but now I'm calculating the weighted average of the 
            # average intensity values so the language is a little confusing)
            interpolated_avg, interpolated_stdev = 0, 0 # Will be filled in with the interpolated values

            R_lower = floor(R)
            R_upper = R_lower + 1
            if R_lower >= len(radial_avgs):
                interpolated_avg = 0 # R value is out of bounds, default to 0
                interpolated_stdev = 0
            elif R_upper >= len(radial_avgs):
                interpolated_avg = radial_avgs[R_lower] # R value is just outside of bounds, default to edge value
                interpolated_stdev = radial_stdevs[R_lower]
            else:
                # Calculate weighted average
                interpolated_avg = (R - R_lower) * radial_avgs[R_upper] + (R_upper - R) * radial_avgs[R_lower]
                interpolated_stdev = (R - R_lower) * radial_stdevs[R_upper] + (R_upper - R) * radial_stdevs[R_lower]
            
            if abs(image[Y][X] - interpolated_avg) <= 3*interpolated_stdev:
                # Value is within acceptance range, do not change
                new_image[Y][X] = image[Y][X] 
            else:
                # Value is outside acceptance range, use average value instead
                new_image[Y][X] = interpolated_avg
    
    return new_image

plt.imshow(SIMULATED_IMAGE)
plt.title("Simulated image")
plt.figure()
plt.imshow(clean_up_image(SIMULATED_IMAGE))
plt.title("Cleaned up image")
plt.show()
